In [272]:
import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:

#CARGAR EL ARCHIVO CON LOS DATOS DE LA AGENCIA 14
ruta_agencia14 = r"C:\Users\sgarcial\Documents\ETB\Agencia 1\SABANA TEMPORAL\DatosAgencia_Enero.xlsx"
df = pd.read_excel(
    ruta_agencia14,
    usecols=['Telefono', 'Nivel_1', 'Franja','NombreServicio', 'Fuente', 'Venta',  'Nivel3'],
    sheet_name='Datos'
)

#CARGAR ARCHIVO CRITERIOS DE LA AGENCIA 14
ruta_criterios = r"C:\Users\sgarcial\Documents\ETB\Agencia 1\Analisis reinyecciones 960014 Agencia Dg_V2 (1).xlsx"
df_criterios = pd.read_excel(
    ruta_criterios,
    sheet_name="Criterios",
    header=None,
)

df_criterios_telefono = df_criterios.copy()

In [ ]:
#APLICAR FILTROS A LAS COLUMNAS NIVEL1, NOMBRE SERVICIO Y VENTA

#FILTRO A LA COLUMNA NIVEL 1
valores_excluir_Nivel1 = [
    "ATENCIÓN",
    "ATENCIÓN, CONTACTO NO UTIL",
    "ATENCIÓN, EN PROCESO",
    "ATENCIÓN, NO VENTA",
    "ATENCIÓN, SERVICIO AL CLIENTE",
    "ATENCIÓN, VENTA HOGARES",
    "SERVICIO AL CLIENTE",
    "VENTA",
    "VENTA FTTH + MÓVIL",
    "VENTA HOGARES",
    "VENTA MOVIL",
    "VENTA NEGOCIO",
    "VENTA NEGOCIOS"]
df["Nivel_1"] = df["Nivel_1"].astype(str).str.strip().str.upper()
df = df[~df["Nivel_1"].isin(valores_excluir_Nivel1)]

#FILTRO A LA COLUMNA NOMBRE SERVICIO
valores_excluir_servicio = [
    "AGENCIA ETB PD REINYECCION",
    "AGENCIA ETB PT",
    "AGENCIA ETB PU",
    "BBDD_IR",
    "REFERIDOS_C2C_2"]
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df = df[~df["NombreServicio"].isin(valores_excluir_servicio)]

#FILTRO A LA COLUMNA VENTA
df["Venta"] = pd.to_numeric(df["Venta"], errors="coerce")
df = df[df["Venta"] == 0]



In [ ]:
# Supón que tu df ya existe y la columna a transformar se llama 'col'
diccionario_franjas = {
    '9:00':  [0, 30, 100, 130, 200, 230, 300, 330, 400, 430, 500, 530, 600, 630, 700, 730, 800, 830],
    '12:00': [900, 930, 1000, 1030, 1100, 1130, 1200, 1230],
    '16:00': [1300, 1330, 1400, 1430, 1500, 1530, 1600, 1630],
    '20:00': [1700, 1730, 1800, 1830, 1900, 1930, 2000, 2030, 2100, 2130, 2200, 2230, 2300, 2330],
}

inv = {v: k for k, vals in diccionario_franjas.items() for v in vals}

# Supón que tu df ya existe y la columna a transformar se llama 'col'
df['Franja'] = df['Franja'].map(inv)

In [ ]:
#CREAR COLUMNA TELEFONO2
df["TELEFONO2"] = (
    df["Telefono"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)
    .str[-10:]
)
df = df.drop_duplicates(subset=["TELEFONO2"])
df = df.drop(columns=["Telefono"])



In [ ]:
#CREAR COLUMNA CATEGORIA, ANTERIORMENTE =BUSCARV(Nombre Servicio; &#39; Análisis de reinyección Hoja1&#39;!A:B; 2; 0)
df_criterios_categoria = df_criterios.copy()
df_criterios_categoria = df_criterios_categoria.iloc[:, [0, 1]]
df_criterios_categoria.columns = ["NombreServicio", "CATEGORIA"]
#df_criterios_categoria = df_criterios_categoria.dropna(subset=["NombreServicio"])
df["NombreServicio"] = df["NombreServicio"].astype(str).str.strip().str.upper()
df_criterios_categoria["NombreServicio"] = df_criterios_categoria["NombreServicio"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios_categoria,
    on="NombreServicio",
    how="left"
)

df2 = df.copy()

In [ ]:
#CREAR COLUMNA FUENTE, ANTERIORMENTE =BUSCARV(Fuente; &#39; Análisis de reinyección Hoja3&#39;!G:K; 5; 0)

# CARGAR LA HOJA CON EL RANGO
df_fuente = pd.read_excel(
    ruta_criterios,
    sheet_name="ANALISIS EFECTIVIDAD",
    header=None,
    engine="openpyxl"
)

df_criterios_fuente_telefono = df_fuente.copy()


# G = 6, K = 10 → 6:11
df_fuente = df_fuente.iloc[:, 6:11]

df_fuente.columns = [
    "Fuente",     # Columna G → CLAVE
    "COL_H",
    "COL_I",
    "COL_J",
    "FUENTE"      # Columna K → RESULTADO BUSCARV
]


df["Fuente"] = df["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()


df = df.merge(
    df_fuente[["Fuente", "FUENTE"]],
    on="Fuente",
    how="left"
)


In [ ]:
#CREAR COLUMNA TIPIFICACION, ANTERIORMENTE =BUSCARV(Nivel_3; &#39;Análisis de reinyección Hoja1&#39;!D:F; 2; 0)
# D = 3, F = 5 → 3:6
df_criterios = df_criterios.iloc[:129, 3:6]

df_criterios.columns = [
    "Nivel3",        # Columna D → CLAVE
    "TIPIFICACION",  # Columna E → RESULTADO BUSCARV
    "COL_F"]

#df_criterios = df_criterios.dropna(subset=["Nivel3"])
df["Nivel3"] = df["Nivel3"].astype(str).str.strip().str.upper()
df_criterios["Nivel3"] = df_criterios["Nivel3"].astype(str).str.strip().str.upper()

df = df.merge(
    df_criterios[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)


In [ ]:
#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "1evd-Dbx3a1o9vONycdM3lcIHcDHtCFk0"
sheet_name = "CONSOLIDADO%20WPP"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"


# 1) Leer Google Sheets (✓ correcto)
df_CONSOLIDADO_WPP_agencia1 = pd.read_csv(
    url,
    usecols=["21"],
    dtype={"21": str}
)
df_CONSOLIDADO_WPP_agencia1 = df_CONSOLIDADO_WPP_agencia1.rename(
    columns={df_CONSOLIDADO_WPP_agencia1.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)

# 2) Normalizar (agregar esta parte que faltaba)
df["TELEFONO2"] = df["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_WPP_agencia1["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()
)

# 3) Merge CORREGIDO (lookup como BUSCARV)
df = df.merge(
    df_CONSOLIDADO_WPP_agencia1.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS con los teléfonos encontrados
df["YAREINYECTADOS"] = df["TELEFONOS_GOOGLE_SHEETS"]
df.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)



In [ ]:
print(df_CONSOLIDADO_WPP_agencia1)

    TELEFONOS_GOOGLE_SHEETS
0                3006108867
1                3057125239
2                3115717739
3                3133146282
4                3054754885
..                      ...
451              3132921736
452              3014076021
453              3145003730
454              3125422858
455              3114858418

[456 rows x 1 columns]


In [ ]:
df_chat_14 = df.copy()

df_chat_14 = df_chat_14[df_chat_14['CATEGORIA'] == 'CHAT']
df_chat_14['FUENTE'] = df_chat_14['FUENTE'].fillna('BAJA')
df_chat_14 = df_chat_14[
    (df_chat_14['TIPIFICACION'] == 'TENER EN CUENTA') | 
    (df_chat_14['TIPIFICACION'] == 'OK')
]

df_chat_14 = df_chat_14[df_chat_14['YAREINYECTADOS'].isna()]
df_chat_14 = df_chat_14[['TELEFONO2', 'FUENTE', 'Franja', 'CATEGORIA']]

df_chat_14 = df_chat_14.drop_duplicates(subset=["TELEFONO2"], keep="first")

df_chat_14["TELEFONO2"] = pd.to_numeric(df_chat_14["TELEFONO2"], errors="coerce")
df_chat_14 = df_chat_14[(df_chat_14["TELEFONO2"] >= 3000000000) & (df_chat_14["TELEFONO2"] <= 3999999999)]



In [ ]:
df_chat_14.sample(10)

,TELEFONO2,FUENTE,Franja,CATEGORIA
2331,3174099217,ALTA,20:00,CHAT
2180,3115130107,BAJA,20:00,CHAT
15684,3142921956,BAJA,9:00,CHAT
14068,3108888485,BAJA,12:00,CHAT
12526,3007221359,ALTA,16:00,CHAT
2305,3215845155,BAJA,20:00,CHAT
8998,3192300236,BAJA,12:00,CHAT
10166,3506221342,ALTA,20:00,CHAT
16209,3153298336,BAJA,12:00,CHAT
10873,3247240428,BAJA,16:00,CHAT


In [ ]:
df_chat_14['CANAL'] = None

for franja, idx in df_chat_14.groupby('Franja').groups.items():
    # idx YA son índices válidos del df
    n = len(idx)
    n_inbox = int(n * 0.7)
    canales = np.array(['INBOX'] * n_inbox + ['CHAT'] * (n - n_inbox))
    np.random.shuffle(canales)
    df_chat_14.loc[idx, 'CANAL'] = canales


In [ ]:
df_chat_14.shape

(332, 5)

In [ ]:
# Suponiendo que tu df principal se llama df_chat_14
# y tiene las columnas 'Franja' y 'CANAL'

df_900_inbox  = df_chat_14[(df_chat_14['Franja'] == '9:00')  & (df_chat_14['CANAL'] == 'INBOX')]
df_900_chat   = df_chat_14[(df_chat_14['Franja'] == '9:00')  & (df_chat_14['CANAL'] == 'CHAT')]

df_1200_inbox = df_chat_14[(df_chat_14['Franja'] == '12:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_1200_chat  = df_chat_14[(df_chat_14['Franja'] == '12:00') & (df_chat_14['CANAL'] == 'CHAT')]

df_1600_inbox = df_chat_14[(df_chat_14['Franja'] == '16:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_1600_chat  = df_chat_14[(df_chat_14['Franja'] == '16:00') & (df_chat_14['CANAL'] == 'CHAT')]

df_2000_inbox = df_chat_14[(df_chat_14['Franja'] == '20:00') & (df_chat_14['CANAL'] == 'INBOX')]
df_2000_chat  = df_chat_14[(df_chat_14['Franja'] == '20:00') & (df_chat_14['CANAL'] == 'CHAT')]


In [ ]:
# Fecha actual en formato ddmmyyyy
fecha = datetime.now().strftime('%d%m%Y')

# Diccionario con todos los 8 df (4 CHAT + 4 INBOX)
todos_dfs = {
    'CHAT': {
        '09_00_00': df_900_chat,
        '12_00_00': df_1200_chat,
        '16_00_00': df_1600_chat,
        '20_00_00': df_2000_chat
    },
    'INBOX': {
        '09_00_00': df_900_inbox,
        '12_00_00': df_1200_inbox,
        '16_00_00': df_1600_inbox,
        '20_00_00': df_2000_inbox
    }
}

# Rutas base
ruta_chat = r'C:\Users\sgarcial\Documents\ETB\Agencia 1\CHAT'
ruta_inbox = r'C:\Users\sgarcial\Documents\ETB\Agencia 1\INBOX'

# Guardar cada df con las columnas requeridas
for canal, dfs_canal in todos_dfs.items():
    ruta = ruta_chat if canal == 'CHAT' else ruta_inbox
    
    for hora, df in dfs_canal.items():
        # Crear nuevo df con las 5 columnas requeridas
        df_salida = pd.DataFrame({
        'phone': ('57' + df['TELEFONO2'].astype(str)).astype('int64'),
        'email': 'evios@envios.com',
        'first_name': '',
        'last_name': '',
        'extra_param': '',
        'format_example': ''
        })


        
        nombre_archivo = f"{fecha}_{canal}_DIRIGIDA_{hora}.xlsx"
        ruta_completa = f"{ruta}\\{nombre_archivo}"
        df_salida.to_excel(ruta_completa, index=False)
        print(f"Guardado: {nombre_archivo}")


Guardado: 06012026_CHAT_DIRIGIDA_09_00_00.xlsx
Guardado: 06012026_CHAT_DIRIGIDA_12_00_00.xlsx
Guardado: 06012026_CHAT_DIRIGIDA_16_00_00.xlsx
Guardado: 06012026_CHAT_DIRIGIDA_20_00_00.xlsx
Guardado: 06012026_INBOX_DIRIGIDA_09_00_00.xlsx
Guardado: 06012026_INBOX_DIRIGIDA_12_00_00.xlsx
Guardado: 06012026_INBOX_DIRIGIDA_16_00_00.xlsx
Guardado: 06012026_INBOX_DIRIGIDA_20_00_00.xlsx


In [ ]:
df2.head()

,Franja,Nivel_1,NombreServicio,Fuente,Venta,Nivel3,TELEFONO2,CATEGORIA
0,9:00,NO VENTA,C2C_2,0021ETBBogotaEx,0,SIN COBERTURA,3123709274,LLAMADA
1,20:00,CONTACTO NO ÚTIL,C2C_2,003GenericoInternetFr,0,LLAMADA NO SERIA,3213983217,LLAMADA
2,9:00,NO VENTA,C2C_2,003GenericoInternetFr,0,SIN COBERTURA,3227541418,LLAMADA
3,9:00,ILOCALIZABLES,C2C_2,0023ETBInternetEx,0,Teléfono no existe,3239553763,LLAMADA
4,20:00,EN PROCESO,C2C_2,003GenericoFibraFr,0,TRAM PENDIENTES (PROV Q ATIENDE),3209859115,LLAMADA


In [ ]:
#CREAR LAS COLUMNAS PARA CATEGORIA == TELEFONO

# 1) CREAR COLUMNA FUENTE CON LOS RANGOS DE TELEFONO
df_fuente = df_criterios_fuente_telefono.iloc[:121, 11:16]  # L:P, filas 32-152 ✓
df_fuente.columns = ["Fuente", "COL_M", "COL_N", "COL_O", "FUENTE"]  # P = FUENTE ✓
df2["Fuente"] = df2["Fuente"].astype(str).str.strip().str.upper()
df_fuente["Fuente"] = df_fuente["Fuente"].astype(str).str.strip().str.upper()
df2 = df2.merge(df_fuente[["Fuente", "FUENTE"]], on="Fuente", how="left")


In [ ]:
# CREAR COLUMNA TIPIFICACION CON EL RANGO DE TELEFONOS

df_criterios_telefono = df_criterios_telefono.iloc[:125, 7:10]

df_criterios_telefono.columns = [
    "Nivel3",        # Columna H → CLAVE
    "TIPIFICACION",  # Columna I → RESULTADO BUSCARV (2ª columna)
    "COL_J"
]

# Normalización para df2 (TELEFONO)
df2["Nivel3"] = df2["Nivel3"].astype(str).str.strip().str.upper()
df_criterios_telefono["Nivel3"] = df_criterios_telefono["Nivel3"].astype(str).str.strip().str.upper()

# Merge para df2
df2 = df2.merge(
    df_criterios_telefono[["Nivel3", "TIPIFICACION"]],
    on="Nivel3",
    how="left"
)

In [ ]:


#LEER ARCHIVO GOOGLE SHEETS Reinyeccion_Agencia_1 PARA EXTRAER TELEFONOS Y CREAR COLUMNA YAREINYECTADOS
sheet_id = "1evd-Dbx3a1o9vONycdM3lcIHcDHtCFk0"
sheet_name_voz = "CONSOLIDADO_VOZ"

# NUEVA URL para hoja VOZ
url_voz = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_voz}"

# 1) Leer Google Sheets (igual que el que funciona)
df_CONSOLIDADO_VOZ_agencia1 = pd.read_csv(
    url_voz,
    usecols=[0],  # ← Columna A (igual que "21" pero índice 0)
    dtype=str
)
df_CONSOLIDADO_VOZ_agencia1 = df_CONSOLIDADO_VOZ_agencia1.rename(
    columns={df_CONSOLIDADO_VOZ_agencia1.columns[0]: "TELEFONOS_GOOGLE_SHEETS"}
)

# 2) Normalizar para df2
df2["TELEFONO2"] = df2["TELEFONO2"].astype(str).str.strip()
df_CONSOLIDADO_VOZ_agencia1["TELEFONOS_GOOGLE_SHEETS"] = (
    df_CONSOLIDADO_VOZ_agencia1["TELEFONOS_GOOGLE_SHEETS"].astype(str).str.strip()
)

# 3) Merge para df2
df2 = df2.merge(
    df_CONSOLIDADO_VOZ_agencia1.drop_duplicates(subset="TELEFONOS_GOOGLE_SHEETS"),
    left_on="TELEFONO2",
    right_on="TELEFONOS_GOOGLE_SHEETS",
    how="left"
)

# 4) Crear YAREINYECTADOS para df2
df2["YAREINYECTADOS"] = df2["TELEFONOS_GOOGLE_SHEETS"]
df2.drop(columns=["TELEFONOS_GOOGLE_SHEETS"], inplace=True)

df2['CATEGORIA'].str.replace(" ", "", regex=False)



0       LLAMADA
1       LLAMADA
2       LLAMADA
3       LLAMADA
4       LLAMADA
         ...   
5563    LLAMADA
5564    LLAMADA
5565    LLAMADA
5566    LLAMADA
5567    LLAMADA
Name: CATEGORIA, Length: 5568, dtype: object

In [ ]:

# APLICAR FILTROS
df_llamada_agencia_1 = df2.copy()

df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['CATEGORIA'] == 'LLAMADA']
df_llamada_agencia_1['FUENTE'] = df_llamada_agencia_1['FUENTE'].fillna('BAJA')
df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['TIPIFICACION'] == 'OK']

df_llamada_agencia_1 = df_llamada_agencia_1[df_llamada_agencia_1['YAREINYECTADOS'].isna()]
df_llamada_agencia_1 = df_llamada_agencia_1[['TELEFONO2', 'FUENTE',]]

df_llamada_agencia_1 = df_llamada_agencia_1.drop_duplicates(subset=["TELEFONO2"], keep="first")

df_llamada_agencia_1["TELEFONO2"] = pd.to_numeric(df_llamada_agencia_1["TELEFONO2"], errors="coerce")
df_llamada_agencia_1 = df_llamada_agencia_1[(df_llamada_agencia_1["TELEFONO2"] >= 3000000000) & (df_llamada_agencia_1["TELEFONO2"] <= 3999999999)]





In [ ]:
df_llamada_agencia_1.shape

(0, 2)